In [283]:
import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd

In [ ]:
crawled_data = pd.read_csv('인공지능_문화체육관광부_클라썸_crawled.csv')

In [284]:
query = '인공지능'
business = '클라썸'

In [285]:
### crawling_trg에 business 이름이 있는 것과 없는 것을 구분하여 데이터를 나눕니다.

crawled_data['checker'] = crawled_data.crawling_trg.str.find(business)

bussiness_news = crawled_data[crawled_data.checker > -1].copy().reset_index(drop=True)

org_news = crawled_data[crawled_data.checker == -1].copy().reset_index(drop=True)
org_news['idx_original'] = range(len(org_news))

business_news_contents = bussiness_news[['crawling_trg', 'pubDate', 'title', 'content', 'link']]
org_news_contents = org_news[['idx_original', 'crawling_trg', 'pubDate', 'title', 'content', 'link']]

In [286]:
### 모델과 토크나이저를 SentenceTransformer 클래스로 불러옵니다.

pretrained_model_path = "sbert/training_klue_sts_klue-roberta-base-2022-08-17_23-27-13"
model = SentenceTransformer(pretrained_model_path)

In [287]:
### 유사도를 계산하여 인덱스를 반환하는 함수입니다.

def retrieve(query, news_contents, top_k):
    query_embedding = model.encode(query)
    news_embeddings = model.encode(news_contents.content)

    cos_scores = util.pytorch_cos_sim(query_embedding, news_embeddings).squeeze()

    top_k = min(top_k, len(news_contents.content))
    top_k_results = torch.topk(cos_scores, k=top_k)

    return top_k_results.indices

In [288]:
### 쿼리와 business 문서 뭉치 간의 유사도 점수를 계산하여 가장 높은 점수의 business 문서를 찾아옵니다.

top_k_results = int(retrieve(query, business_news_contents, 1))
top_of_business_news_contents = business_news_contents.iloc[top_k_results]

In [289]:
### 기관별로 데이터를 나눕니다.

org_news_contents_splits = []
for org in org_news_contents.crawling_trg.unique():
    org_news_contents_split = org_news_contents[org_news_contents.crawling_trg == org].reset_index(drop=True).copy()
    org_news_contents_splits.append(org_news_contents_split)

In [290]:
### 쿼리와 각 기관별 문서 뭉치간의 유사도 점수를 계산하여 기관별 가장 높은 점수의 문서를 찾아옵니다.

tops_of_org_news_contents = []
for org_news_contents_split in org_news_contents_splits:
    top_k_results = int(retrieve(query, org_news_contents_split, 1))
    tops_of_org_news_contents.append(org_news_contents_split.iloc[top_k_results])
    
tops_of_org_news_contents = list(map(lambda x: x.to_list(), tops_of_org_news_contents))
tops_of_org_news_contents = pd.DataFrame(tops_of_org_news_contents, columns=['idx_original', 'crawling_trg', 'pubDate', 'title', 'content', 'link'])

In [291]:
### 가장 높은 점수의 business 문서를 쿼리로 하여 
### 가장 높은 점수의 기관별 문서 뭉치와 유사도 점수를 계산하고 유사도 점수 상위 5개 문서를 찾아옵니다.

top_k_results = retrieve(top_of_business_news_contents.content, tops_of_org_news_contents, 5)
result = tops_of_org_news_contents.iloc[top_k_results]

,idx_original,crawling_trg,pubDate,title,content,link
2,46,국립아시아문화전당 인공지능,"Tue, 27 Sep 2022 19:45:00 +0900",[키워드 브리핑] AICON 광주 2022 등,"- AICON 광주 2022- '2022 청소년과학페어 전국대회', 국제 워크숍 해...",https://n.news.naver.com/mnews/article/660/000...
20,253,한국문학번역원 인공지능,"Thu, 18 Aug 2022 02:29:00 +0900",8월18일 알림,국보 제332호 수마노탑. 정암사 제공 ◇ 정선 정암사는(주지 천웅 스님)는 18일...,https://n.news.naver.com/mnews/article/028/000...
4,57,국립중앙도서관 인공지능,"Sun, 16 Oct 2022 10:02:00 +0900",[2022 기사창투 주역-③]한국전자통신연구원 호남권연구센터,한국전자통신연구원(ETRI) 호남권연구센터는 광주·전남·전북·제주 등 호남광역경제권...,https://n.news.naver.com/mnews/article/030/000...
26,393,영화진흥위원회 영화관입장권 통합전산망 인공지능,"Mon, 08 Aug 2022 11:09:00 +0900","[ET라씨로] 개봉 D-2 헌트 예매율 1위…버킷스튜디오, 주가 18% ↑",ET라씨로는 인공지능(AI) 기반으로 선별·분석한 주식 정보와 종목 매매 신호를 제...,https://n.news.naver.com/mnews/article/030/000...
16,163,영화진흥위원회 인공지능,"Mon, 08 Aug 2022 11:09:00 +0900","[ET라씨로] 개봉 D-2 헌트 예매율 1위…버킷스튜디오, 주가 18% ↑",ET라씨로는 인공지능(AI) 기반으로 선별·분석한 주식 정보와 종목 매매 신호를 제...,https://n.news.naver.com/mnews/article/030/000...
